# BC COVID-19 Data by Health Authority

1. [Feature Server JSON](https://services1.arcgis.com/xeMpV7tU1t4KD3Ei/arcgis/rest/services/COVID19_Cases_by_BC_Health_Authority/FeatureServer/0?f=pjson) `id 1,2,3,4,5`

2. [Case Details](http://www.bccdc.ca/health-info/diseases-conditions/covid-19/data)

---

In [7]:
import pandas as pd
import geopandas as gpd
import requests
from shapely.geometry import shape

In [3]:
r = requests.get("https://services1.arcgis.com/xeMpV7tU1t4KD3Ei/arcgis/rest/services/COVID19_Cases_by_BC_Health_Authority/FeatureServer/0/query?where=&objectIds=1%2C2%2C3%2C4%2C5&time=&geometry=&geometryType=esriGeometryPolygon&inSR=&spatialRel=esriSpatialRelIntersects&resultType=none&distance=0.0&units=esriSRUnit_Meter&returnGeodetic=false&outFields=&returnGeometry=true&returnCentroid=false&featureEncoding=esriDefault&multipatchOption=xyFootprint&maxAllowableOffset=&geometryPrecision=&outSR=&datumTransformation=&applyVCSProjection=false&returnIdsOnly=false&returnUniqueIdsOnly=false&returnCountOnly=false&returnExtentOnly=false&returnQueryGeometry=true&returnDistinctValues=false&cacheHint=false&orderByFields=&groupByFieldsForStatistics=&outStatistics=&having=&resultOffset=&resultRecordCount=&returnZ=false&returnM=false&returnExceededLimitFeatures=true&quantizationParameters=&sqlFormat=none&f=pgeojson&token=")
r.raise_for_status()

data = r.json()

In [4]:
gdf = gpd.GeoDataFrame.from_features(data["features"])
print(gdf.head())

                                            geometry            HA_Name
0  POLYGON ((-120.45314 52.99326, -120.42102 52.9...           Interior
1  MULTIPOLYGON (((-122.93676 49.31128, -122.9365...             Fraser
2  MULTIPOLYGON (((-123.10903 49.28623, -123.0231...  Vancouver Coastal
3  MULTIPOLYGON (((-126.66725 51.19230, -126.6705...   Vancouver Island
4  MULTIPOLYGON (((-129.76908 55.45115, -129.7657...           Northern


In [5]:
gdf.to_json("../data/bc_covid.json")

ValueError: Unknown na method ../data/bc_covid.json

In [11]:
with open('../data/bc-health-authorities.geojson', 'w') as f:
    f.write(gdf.to_json())

---

2. **Case Details**

In [9]:
pd.read_csv('../data/BCCDC_COVID19_Dashboard_Case_Details.csv')

,Reported_Date,HA,Sex,Age_Group,Classification_Reported
0,2020-01-26,Out of Canada,M,40-49,Lab-diagnosed
1,2020-02-02,Vancouver Coastal,F,50-59,Lab-diagnosed
2,2020-02-05,Out of Canada,F,20-29,Lab-diagnosed
3,2020-02-05,Out of Canada,M,30-39,Lab-diagnosed
4,2020-02-11,Interior,F,30-39,Lab-diagnosed
...,...,...,...,...,...
3295,2020-07-19,Vancouver Island,M,80-89,Lab-diagnosed
3296,2020-07-19,Vancouver Island,F,60-69,Lab-diagnosed
3297,2020-07-19,Vancouver Island,F,50-59,Lab-diagnosed
3298,2020-07-19,Out of Canada,F,30-39,Lab-diagnosed
